# Comparing VBD Outputs: Waymax vs GPUDrive

In [1]:
%%capture
import waymax
import numpy as np
import math
import mediapy
from tqdm import tqdm
import dataclasses
import os
import shutil
from pathlib import Path
import pickle
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from waymax import config as _config
from waymax import dataloader, datatypes, visualization, dynamics
from waymax.datatypes.simulator_state import SimulatorState
from waymax.config import EnvironmentConfig, ObjectType

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

# VBD dependencies
from integrations.models.vbd.sim_agent.waymax_env import WaymaxEnvironment
from integrations.models.vbd.data.dataset import WaymaxTestDataset
from integrations.models.vbd.waymax_visualization.plotting import plot_state
from integrations.models.vbd.sim_agent.sim_actor import VBDTest, sample_to_action
from integrations.models.vbd.model.utils import set_seed

# GPUDrive dependencies
import gpudrive
from pygpudrive.env.config import EnvConfig, RenderConfig, SceneConfig, SelectionDiscipline
from pygpudrive.env.env_torch import GPUDriveTorchEnv
from pygpudrive.visualize.utils import img_from_fig

# Plotting
sns.set("notebook")
sns.set_style("ticks", rc={"figure.facecolor": "none", "axes.facecolor": "none"})
#%config InlineBackend.figure_format = 'svg'

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

2025-01-03 17:02:53.423490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735903973.441764  141796 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735903973.446943  141796 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-03 17:02:53.465272: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Configurations

In [2]:
DATA_DIR = 'data/processed' # Base data path
CKPT_DIR = 'data/checkpoints' # Base checkpoint path
CKPT_PATH = 'integrations/models/vbd/weights/epoch=18.ckpt'

SCENARIO_ID = 'efc5cbe01b4a526f'

FPS = 20
INIT_STEPS = 11 # Warmup period
MAX_CONTROLLED_OBJECTS = 32

### Load pre-trained VBD model

In [3]:
# Load model
model = VBDTest.load_from_checkpoint(CKPT_PATH, torch.device('cpu'))
_ = model.cpu()
_ = model.eval();

# Model settings
replan_freq=80 # Roll out every X steps 80 means openloop
model.early_stop=0 # Stop Diffusion Early From 100 to X
model.skip = 1 # Skip Alpha 
model.reward_func = None

# Ensure reproducability
#set_seed(5)

In [4]:
#Function to save frames side by side.
def save_side_by_side_gif(frames1, frames2, output_path, fps=FPS):
    """
    Saves two arrays of frames as a single side-by-side GIF.

    Parameters:
        frames1 (list of np.ndarray): First array of frames (images).
        frames2 (list of np.ndarray): Second array of frames (images).
        output_path (str): Path to save the output GIF.
        fps (int): Frames per second for the GIF.
    """
    # Ensure both arrays have the same number of frames
    if len(frames1) != len(frames2):
        raise ValueError("The two frame arrays must have the same number of frames.")
    
    # Combine frames side by side
    combined_frames = []
    for frame1, frame2 in zip(frames1, frames2):
        # Ensure frames have the same height
        if frame1.shape[0] != frame2.shape[0]:
            raise ValueError("Frames must have the same height to combine them side by side.")
        
        # Concatenate frames horizontally
        combined_frame = np.hstack((frame1, frame2))
        combined_frames.append(combined_frame)
    
    # Save the combined frames as a GIF
    mediapy.write_video(output_path, combined_frames, fps=fps, codec="gif")
    print(f"GIF saved at {output_path}")

### Make Videos

In [5]:
# Create waymax test "dataset" obj (need for utils)
dataset = WaymaxTestDataset(
    data_dir = 'data/processed/debug/waymax', 
    anchor_path = 'integrations/models/vbd/data/cluster_64_center_dict.pkl',
    max_object=MAX_CONTROLLED_OBJECTS,
)
#Init Waymax env
env_config = EnvironmentConfig(
    controlled_object=ObjectType.VALID,
    allow_new_objects_after_warmup=False,
    init_steps=INIT_STEPS+1,
    max_num_objects=MAX_CONTROLLED_OBJECTS,
)
waymax_env = WaymaxEnvironment(
    dynamics_model=dynamics.StateDynamics(),
    config=env_config,
    log_replay = True,
)

with open(f'{DATA_DIR}/debug/waymax/waymax_scenario_{SCENARIO_ID}.pkl', 'rb') as f:
    scenario = pickle.load(f)

#Generate video with vbd trajectories
init_state = waymax_env.reset(scenario)
current_state = init_state
sample = dataset.process_scenario(
init_state,
current_index=init_state.timestep,
use_log=False
)
is_controlled = sample['agents_interested'] > 0
selected_agents = sample['agents_id'][is_controlled]
state_logs = [current_state]

for i in range(current_state.remaining_timesteps):
    t = i % replan_freq
    if t == 0:
        sample = dataset.process_scenario(
        current_state, 
        current_index = current_state.timestep,
        use_log=False,
        selected_agents=selected_agents, # override the agent selection by distance to the ego
        )
        batch = dataset.__collate_fn__([sample])
        pred = model.sample_denoiser(batch)
        traj_pred = pred['denoised_trajs'].cpu().numpy()[0]

    # Get action
    action_sample = traj_pred[:, t, :]
    action = sample_to_action(
    action_sample, 
    is_controlled, 
    agents_id=selected_agents, 
    max_num_objects=MAX_CONTROLLED_OBJECTS
    )
    # Step the simulator
    current_state = waymax_env.step_sim_agent(current_state, [action])
    state_logs.append(current_state)

waymax_frames = [plot_state(state) for state in state_logs]

#Init GPUDrive env
env_config = EnvConfig(
    init_steps=INIT_STEPS, # Warmup period
    remove_non_vehicles=False, # Control vehicles, pedestrians, and cyclists
    return_vbd_data=True, # Use VBD
    dynamics_model="state", # Use state-based dynamics model
    dist_to_goal_threshold=1e-5, # Trick to make sure the agents don't disappear when they reach the goal
    collision_behavior="ignore", # Ignore collisions
    init_all_objects = True
)
        
# Make env
gpudrive_env = GPUDriveTorchEnv(
    config=env_config,
    scene_config=SceneConfig(path="data/processed/temp", num_scenes=1),
    render_config=RenderConfig(draw_obj_idx=True, render_init=True, resolution=(400, 400)),
    max_cont_agents=MAX_CONTROLLED_OBJECTS, # Maximum number of agents to control per scene
    device="cpu",
)

#Generate video with VBD trajectories 
init_state = gpudrive_env.reset()
selected_agents = torch.nonzero(gpudrive_env.cont_agent_mask[0, :]).flatten().tolist()
# Obtain all info for diffusion model (warmup)
gpudrive_sample_batch = gpudrive_env.sample_batch
# Obtain predicted trajectories
pred = model.sample_denoiser(gpudrive_sample_batch)#, x_t=x_t)
vbd_traj_pred = pred['denoised_trajs'].cpu().numpy()[0]
is_controlled = gpudrive_sample_batch['agents_interested'] > 0
agent_indices = gpudrive_sample_batch['agents_id'][0]

pred_trajs = torch.zeros((gpudrive_env.max_agent_count, env_config.episode_len-INIT_STEPS, 10))
pred_trajs[agent_indices, :, :2] = torch.Tensor(vbd_traj_pred[:, :, 0:2]) # pos x, y
pred_trajs[agent_indices, :, :2] -= gpudrive_env.sim.world_means_tensor().to_torch()[0, :2] #re-mean the predicted trajectory positions
pred_trajs[agent_indices, :, 3] = torch.Tensor(vbd_traj_pred[:, :, 2]) # yaw 
pred_trajs[agent_indices, :, 4:6] = torch.Tensor(vbd_traj_pred[:, :, 3:5]) # vel x, y
pred_trajs = pred_trajs.unsqueeze(0)

gpudrive_frames = []

# Step
for t in range(env_config.episode_len-INIT_STEPS):
    gpudrive_env.step_dynamics(pred_trajs[:, :, t, :])
    fig = gpudrive_env.vis.plot_simulator_state(
        env_indices=[0],
        time_steps=[t],
        figsize = (4, 4),
        zoom_radius=70,
    )[0]
    
    gpudrive_frames.append(img_from_fig(fig))

save_side_by_side_gif(waymax_frames, gpudrive_frames, f'integrations/models/videos/{SCENARIO_ID}.gif')


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Found 55 scenarios


Diffusion: 100%|██████████| 50/50 [00:17<00:00,  2.92it/s]



--- Ratio unique scenes / number of worls =         1 / 1 ---



Diffusion: 100%|██████████| 50/50 [00:13<00:00,  3.76it/s]


GIF saved at integrations/models/videos/efc5cbe01b4a526f.gif
